## 使用 PyTorch 内置的线性回归

In [28]:
import torch.nn as nn

In [29]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### 加载数据

In [30]:
from torch.utils.data import TensorDataset

In [31]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

### 制作数据集

In [32]:
from torch.utils.data import DataLoader

In [37]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [38]:
# 数据加载器通常搭配 for-in 循环使用
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.]])
tensor([[ 56.,  70.],
        [ 22.,  37.],
        [103., 119.],
        [103., 119.],
        [ 56.,  70.]])


## nn.Linear

In [39]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.2121, -0.3805, -0.2948],
        [ 0.0823,  0.3428, -0.2692]], requires_grad=True)
Parameter containing:
tensor([0.5119, 0.5020], requires_grad=True)


In [40]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2121, -0.3805, -0.2948],
         [ 0.0823,  0.3428, -0.2692]], requires_grad=True),
 Parameter containing:
 tensor([0.5119, 0.5020], requires_grad=True)]

In [41]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-22.1680,  17.9017],
        [-32.5292,  20.9285],
        [-49.1100,  37.9845],
        [ -5.1172,  13.6755],
        [-42.0079,  20.2455],
        [-22.1680,  17.9017],
        [-32.5292,  20.9285],
        [-49.1100,  37.9845],
        [ -5.1172,  13.6755],
        [-42.0079,  20.2455],
        [-22.1680,  17.9017],
        [-32.5292,  20.9285],
        [-49.1100,  37.9845],
        [ -5.1172,  13.6755],
        [-42.0079,  20.2455]], grad_fn=<AddmmBackward>)

## Loss Func

In [42]:
# Import nn.functional
import torch.nn.functional as F

In [43]:
# Define loss function
loss_fn = F.mse_loss

In [44]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(9747.2871, grad_fn=<MseLossBackward>)


## Optimizer

In [51]:
# Define optimizer
# opt = torch.optim.SGD(model.parameters(), lr=1e-5)
opt = torch.optim.Adam(model.parameters(), lr=1e-5)

In [52]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [53]:
fit(200, model, loss_fn, opt)

Epoch [10/200], Loss: 17.4223
Epoch [20/200], Loss: 25.2060
Epoch [30/200], Loss: 26.5489
Epoch [40/200], Loss: 16.8075
Epoch [50/200], Loss: 14.0220
Epoch [60/200], Loss: 26.5221
Epoch [70/200], Loss: 28.3982
Epoch [80/200], Loss: 29.7304
Epoch [90/200], Loss: 13.9761
Epoch [100/200], Loss: 24.5525
Epoch [110/200], Loss: 25.8716
Epoch [120/200], Loss: 11.4839
Epoch [130/200], Loss: 25.1122
Epoch [140/200], Loss: 28.2930
Epoch [150/200], Loss: 29.6210
Epoch [160/200], Loss: 16.0239
Epoch [170/200], Loss: 28.9794
Epoch [180/200], Loss: 15.2280
Epoch [190/200], Loss: 30.2855
Epoch [200/200], Loss: 17.3388


In [54]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.7489,  70.9901],
        [ 80.0942,  96.9249],
        [122.6442, 140.5514],
        [ 23.6586,  39.9260],
        [ 96.8359, 111.0109],
        [ 57.7489,  70.9901],
        [ 80.0942,  96.9249],
        [122.6442, 140.5514],
        [ 23.6586,  39.9260],
        [ 96.8359, 111.0109],
        [ 57.7489,  70.9901],
        [ 80.0942,  96.9249],
        [122.6442, 140.5514],
        [ 23.6586,  39.9260],
        [ 96.8359, 111.0109]], grad_fn=<AddmmBackward>)

In [55]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [57]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3117,  0.9479,  0.3834],
         [-0.1734,  0.9853,  0.3983]], requires_grad=True),
 Parameter containing:
 tensor([0.5119, 0.5046], requires_grad=True)]